<center>
    <h1>Buying Happiness</h1>

    <h2>Turning Words Into Numbers for Maximum Utility</h2>
    <img src="./images/logo-q-press-logo-red.png" alt="Quantopian" style="width: 200px;"/>
    
    <h4>Max Margenot</h4>
    <h4>Lead - Data Science/Academia</h4>
</center>

*This presentation is for informational purposes only and does not constitute an offer to sell, a solicitation to buy, or a recommendation for any security; nor does it constitute an offer to provide investment advisory or other services by Quantopian, Inc. ("Quantopian"). Nothing contained herein constitutes investment advice or offers any opinion with respect to the suitability of any security, and any views expressed herein should not be taken as advice to buy, sell, or hold any security or as an endorsement of any security or company. In preparing the information contained herein, Quantopian, Inc. has not taken into account the investment needs, objectives, and financial circumstances of any particular investor. Any views expressed and data illustrated herein were prepared based upon information, believed to be reliable, available to Quantopian, Inc. at the time of publication. Quantopian makes no guarantees as to their accuracy or completeness. All information is subject to change and may quickly become unreliable for various reasons, including changes in market conditions or economic circumstances.*

# Machine Learning

* Gathers insight on models from other models and from data
* You can learn a style of trading from past sets of trades

* Use data to come up with a new style of trading
* Unstructured data as a source of insight

## Natural Language Processing (NLP)

* Computer science and linguistics tools to interpret human language and text
* Allows us to quantify unstructured text

## Data Processing

* Stopwords
* Tokenization
* Stemming

## Feature Generation

### Bag of Words

#### Unigrams

In [13]:
my_phrases = ['Show me the alpha', 'Carthago delenda est',]
[phrase.split(' ') for phrase in my_phrases]

[['Show', 'me', 'the', 'alpha'], ['Carthago', 'delenda', 'est']]

#### Bigrams

In [10]:
[word for phrase in my_phrases for word in zip(phrase.split(' ')[:-1], phrase.split(' ')[1:])]

[('Show', 'me'),
 ('me', 'the'),
 ('the', 'alpha'),
 ('Carthago', 'delenda'),
 ('delenda', 'est')]

## Feature Generation

### Word Embeddings

In [1]:
import gensim
w2v = gensim.models.Word2Vec.load('../../open_source/sensitivipy/d2v')

Using TensorFlow backend.


In [10]:
w2v.wv['bless']

array([ 2.62551141,  0.4847329 ,  0.45857674,  0.78889996, -0.43683404,
       -0.53521132, -0.42866716, -0.97290128,  4.26054811,  0.16179222,
       -1.51473927, -0.67379916,  0.91109943, -0.52419966, -0.58900422,
       -0.64030105,  0.51280391, -1.69713056,  3.31448555,  1.11600494,
       -0.79634368, -0.33716959, -0.69943726,  1.17677033,  1.7335273 ,
        0.51291114, -0.14284545,  1.29976273,  1.54217029, -0.62447971,
       -2.48501277, -1.00985312, -1.52342713, -1.05349064, -1.50105405,
       -0.53860253, -1.22803974,  3.16734791, -0.75309408,  1.87978327,
       -2.15208626,  0.48014098,  2.36944318,  1.19643748, -1.07344544,
       -0.713144  , -1.06406701, -0.78677535,  0.01792815,  1.49741733,
        0.26484376, -1.14412224,  0.44056687,  1.39928448, -1.02806461,
       -0.95280224, -1.12010801, -0.4622038 ,  4.02090597, -1.2571429 ,
        1.50985861, -1.4511081 , -1.14850509, -0.05078466, -1.58439875,
        1.1981262 , -2.02171135, -2.81897902, -1.1634382 ,  0.04

## Feature Generation

### Word Embeddings

In [22]:
w2v.wv.most_similar('bless')

[('prais', 0.632722020149231),
 ('prayer', 0.5398192405700684),
 ('grate', 0.5342082977294922),
 ('thank', 0.5135688185691833),
 ('pray', 0.4962601065635681),
 ('xoxo', 0.47532156109809875),
 ('humbl', 0.47377634048461914),
 ('faith', 0.4691312313079834),
 ('safe', 0.4633764922618866),
 ('inspir', 0.456495463848114)]

## The Data

<center><img src="./images/twitter_logo.png"></center>

## Sentiment140

* Twitter sentiment dataset

## My Approach

* Logistic Regression with Bag of Words

* Neural Network with Word Embeddings

## Logistic Regression

\begin{eqnarray}
g(X) &=& \alpha + \beta_0 X_0 + \cdots + \beta_n X_N \\
F(X) &=& \frac{1}{1 + e^{-g(X)}}
\end{eqnarray}

* Friendly, familiar linear model
* Easy to interpret and understand

## Results

<img src="./images/is_logistic_model.png" alt="Logistic Regression Accuracy" style="width: 800px;"/>

<img src="./images/oos_logistic_model.png" alt="Logistic Regression Accuracy" style="width: 800px;"/>

## Neural Networks

<center>
<img src="./images/basic_neural_network.png" alt="Neural Network" style="width: 400px;"/>
</center>

## Neural Networks

<center>
<img src="./images/neural_network.png" alt="Neural Network" style="width: 400px;"/>
</center>

## Recurrent Neural Networks

<center>
<img src="./images/recurrent_neural_network.png" alt="Recurrent Neural Network" style="width: 400px;"/>
</center>

## Recurrent Neural Networks

* Recurrent neural networks good for text processing
* View sentences as sequences of words
    * Similar to time series structure

## Long Short-term Memory Networks

<center>
<img src="./images/lstm.svg" alt="Long Short-term Memory Network" style="width: 800px;"/>
</center>

## Long Short-term Memory Networks

* In Keras this is simple to implement

In [ ]:
input_layer = Input(shape=(MAX_WORDS,))
lstm_layer = LSTM(64,
                  dropout=DROPOUT,
                  activation='tanh')(input_layer)
lstm_layer = LSTM(128,
                  dropout=DROPOUT,
                  activation='tanh')(lstm_layer)

## Results

<center>
<img src="./images/nn_training.png" alt="Neural Network Training" style="width: 800px;"/>
</center>

<center>
<img src="./images/presidential_seal.png" alt="USA" style="width: 400px;"/>
</center>

## How to Compute A Signal?

* Hypothesis: POTUS's tweets affect the market

## Precedent

* Trump2Cash: https://github.com/maxbbraun/trump2cash
    * Waits for the president to mention publicly-traded companies
    * Uses sentiment analysis to determine positivity or negativity
    * Enters long or short positions based on sentiment

* BOTUS: https://twitter.com/botus
    * Waits for the president to mention publicly-traded companies
    * Uses sentiment analysis to determine positivity or negativity
    * Enters long or short positions based on sentiment

* Trump & Dump: https://www.t-3.com/work/the-trump-and-dump-bot-analyze-tweets-short-stocks-save-puppies-all-in-seconds
    * Waits for the president to mention publicly-traded companies
    * Uses sentiment analysis to determine positivity or negativity
    * Enters short positions based if sentiment is negative

# Factor Models

* CAPM

$$ r_p = \alpha + \beta_{m} r_{m} $$

* Fama-French Factors

$$ r_p = \alpha + \beta_{m} r_{m} + \beta_{hml} r_{hml} + \beta_{smb} r_{smb} $$

* Alternative Factors

$$ r_p = \alpha + \beta_0 r_0 + \cdots + \beta_n r_n $$

## 30-day Moving Average of Sentiment

<img src="./images/rolling_sentiment.png" alt="Sentiment Score" style="width: 800px;"/>

## Sentiment Exposure Signal

<img src="./images/return_buckets.png" alt="Mean Return Buckets" style="width: 800px;"/>

## Sentiment Exposure Signal

<img src="./images/cumulative_return.png" alt="Cumulative Returns" style="width: 800px;"/>

## Sentiment Exposure Signal

<img src="./images/cumulative_return_buckets.png" alt="Cumulative Return Buckets" style="width: 800px;"/>

## Risk Analysis

<img src="./images/sentiment_beta_risk_exposures.png" alt="Risk Exposures" style="width: 800px;"/>

## Returns Breakdown

<img src="./images/returns_breakdown.png" alt="Common and Specific Returns" style="width: 800px;"/>

<img src="./images/returns_breakdown_cumulative.png" alt="Common and Specific Returns" style="width: 800px;"/>

## Possible Improvements

* Lemmatization instead of Stemming
* Sense2Vec

## Tools

* scikit-learn: https://github.com/scikit-learn/scikit-learn
* Gensim: https://github.com/RaRe-Technologies/gensim
* Keras: https://github.com/fchollet/keras
* Alphalens: https://github.com/quantopian/alphalens
* Pyfolio: https://github.com/quantopian/pyfolio


* Quantopian Research: https://www.quantopian.com/research
* Quantopian Lecture Series: https://www.quantopian.com/lectures

# References

* Go, A., Bhayani, R. and Huang, L., 2009. Twitter sentiment classification using distant supervision. CS224N Project Report, Stanford, 1(2009), p.12.
    * http://www.sentiment140.com/

<div style="text-align:center">
    <center><img src="./images/twitter_logo.png" alt='Twitter' style="width: 50px;"/></center>
    <h3>@clean_utensils</h3>
</div>

<div style="text-align:center">
    <center><img src="./images/github_logo.png" alt="Github" style="width: 50px;"/></center>
    <h3>@mmargenot</h3>
</div>

<div style="text-align:center">
    <center><img src="./images/logo-q-press-logo-red.png" alt="Quantopian" style="width: 100px;"/></center>
    <h3>max@quantopian.com</h3>
</div>

*This presentation is for informational purposes only and does not constitute an offer to sell, a solicitation to buy, or a recommendation for any security; nor does it constitute an offer to provide investment advisory or other services by Quantopian, Inc. ("Quantopian"). Nothing contained herein constitutes investment advice or offers any opinion with respect to the suitability of any security, and any views expressed herein should not be taken as advice to buy, sell, or hold any security or as an endorsement of any security or company. In preparing the information contained herein, Quantopian, Inc. has not taken into account the investment needs, objectives, and financial circumstances of any particular investor. Any views expressed and data illustrated herein were prepared based upon information, believed to be reliable, available to Quantopian, Inc. at the time of publication. Quantopian makes no guarantees as to their accuracy or completeness. All information is subject to change and may quickly become unreliable for various reasons, including changes in market conditions or economic circumstances.*

In [6]:
!jupyter nbconvert --to slides ./Buying_Happiness_Talk.ipynb --reveal-prefix=http://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.2.0

[NbConvertApp] Converting notebook ./Buying_Happiness_Talk.ipynb to slides
[NbConvertApp] Writing 285959 bytes to ./Buying_Happiness_Talk.slides.html
